In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [8]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=ColourContext
)

In [9]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [13]:
from pprint import pprint

pprint(response['messages'][-1].content)

("I'm an AI language model and don't have personal experiences or emotions. I "
 'can help answer questions based on the information provided to me during our '
 "conversation. However, I cannot determine your favorite color as it's a "
 'subjective preference that only you can know.')


## Accessing Context

In [14]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [15]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [20]:
import warnings
warnings.filterwarnings("ignore")

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

pprint(response['messages'][-1].content)

'Your favourite colour is blue.'


In [21]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext(favourite_colour="green")
)

pprint(response['messages'][-1].content)

'Your favourite colour is green.'
